# PyTerrier Starter: Full-Rank with ChatNoir for Task 3 in Touche 2023

This is the [PyTerrier](https://github.com/terrier-org/pyterrier) baseline for [task 3 on Image Retrieval for Arguments](https://touche.webis.de/clef23/touche23-web/image-retrieval-for-arguments.html) in [Touché 2023](https://touche.webis.de/clef23/touche23-web/).

This notebook implements "full-rank" retrieval with BM25 in PyTerrier that indexes all images before retrieval .

### Adapt the notebook locally

You can adapt/run this baseline locally with docker and can directly deploy and run it in [TIRA.io](https://www.tira.io/task/touche-task-3).

With docker installed, you can start this notebook with the command:

```
docker run --rm -ti \
    -p 8888:8888 \
    -v ${PWD}:/workspace \
    webis/tira-touche23-task-3-pyterrier-baseline:0.0.1 \
    jupyter-lab --allow-root --ip 0.0.0.0
```

### Deployment in TIRA

To deploy approaches in TIRA, you upload the image and specify the command that is to be executed in the image. TIRA gives you a personalized documentation on how to upload the image, and to run this notebook in TIRA you can specify the following command in TIRA:

```
/workspace/run-notebook.py --notebook /workspace/full-rank-pipeline.ipynb --input $inputDataset --output $outputDir
```

You can dry-run this on your machine by executing the command:

```
./run-notebook.py \
    --input ${PWD}/sample-input/full-rank \
    --output ${PWD}/sample-output \
    --notebook /workspace/full-rank-pipeline.ipynb \
    --local-dry-run True
```



### Additional Resources

- The [PyTerrier tutorial](https://github.com/terrier-org/ecir2021tutorial)
- The [PyTerrier documentation](https://pyterrier.readthedocs.io/en/latest/)
- The [TIRA quickstart](https://touche.webis.de/clef23/touche23-web/evidence-retrieval-for-causal-questions.html#tira-quickstart)



### Step 1: Import everything and load variables

In [1]:
from tira_utils import get_input_directory_and_output_directory, normalize_run
import pyterrier as pt
import pandas as pd
import os
import json
from tqdm import tqdm
from glob import glob
from pathlib import Path

SYSTEM_NAME = os.environ.get('TIRA_SYSTEM_NAME' ,'my-retrieval-system')

if not pt.started():
    # tira_utils above should already have done started pyterrier with this configuration to ensure that no internet connection is required (for reproducibility)
    pt.init(version=os.environ['PYTERRIER_VERSION'], helper_version=os.environ['PYTERRIER_HELPER_VERSION'], no_download=True)

input_directory, output_directory = get_input_directory_and_output_directory(default_input='/workspace/sample-input/full-rank')


Start PyTerrier with version=5.7, helper_version=0.0.7, no_download=True


PyTerrier 0.9.1 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


I will use a small hardcoded example located in /workspace/sample-input/full-rank.
The output directory is /tmp/


### Step 2: Load the Queries

In [2]:
print('Step 2: Load the queries.')

def load_queries():
    file_name = input_directory + '/queries.jsonl'
    
    if not os.path.exists(file_name):
        raise ValueError(f'Could not find the file "{file_name}". Got: {glob(input_data + "/*")}')
    
    ret = pd.read_json(file_name, lines=True)

    # https://github.com/terrier-org/pyterrier/issues/62\n",
    ret['query'] = ret['query'].apply(lambda i: "".join([x if x.isalnum() else " " for x in i]))
    
    return ret

queries = load_queries()  
queries.head(2)

Step 2: Load the queries.


,qid,query
0,34,Are social networking sites good for our society
1,48,Should the voting age be lowered


### Step 3: Index the images

In [3]:
print('Step 3: Create the Index.')

# We use some very baseline method to get a textual representation: we just use the text of the pages that contain the image.
def load_image_text(image_id):
    ret = ''
    
    for txt_file in glob(input_directory +'/images/' + image_id[:3] + '/' + image_id + '/*/*/*/text.txt'):
        ret += '\n\n' + open(txt_file).read()
        
    return ret.strip()

def all_images():
    for i in glob(input_directory + '/images/*/*'):
        image_id = i.split('/')[-1]
        yield {'docno': image_id, 'text': load_image_text(image_id)}


!rm -Rf ./index
iter_indexer = pt.IterDictIndexer("./index", meta={'docno': 20, 'text': 4096})
index_ref = iter_indexer.index(tqdm(all_images()))


Step 3: Create the Index.


50it [00:01, 38.39it/s]


### Step 5: Define the Pipeline

In [4]:
print('Step 5: Define the Pipeline.')

retrieval_pipeline = pt.BatchRetrieve(index_ref, wmodel="BM25", verbose=True)

Step 5: Define the Pipeline.


### Step 6: Create Run

In [5]:
print('Step 6: Create Run.')
run = retrieval_pipeline(queries)

Step 6: Create Run.


BR(BM25): 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  4.80q/s]


In [6]:
run.head(3)

,qid,docid,docno,rank,score,query
0,34,39,Iad17912610912ffd,0,-0.152528,Are social networking sites good for our society
1,34,33,Ia20c1e2e90f832cb,1,-0.307966,Are social networking sites good for our society
2,34,49,I374eede3492beb08,2,-1.046229,Are social networking sites good for our society


### Step 7: Stence Detection

In [7]:
print('Step 7: Define stence detection')

def detect_stance(query_document_pair):
    # As baseline, we return always pro
    return 'PRO'

run['Q0'] = run.apply(lambda i: detect_stance(i), axis=1)

Step 7: Define stence detection


### Step 6: Persist Run

In [8]:
print('Step 6: Persist Run.')

Path(output_directory).mkdir(parents=True, exist_ok=True)
normalize_run(run, SYSTEM_NAME).to_csv(output_directory + '/run.txt', sep=' ', header=False, index=False)

print('Done...')

Step 6: Persist Run.
Done...
